# **Créditos por porcentaje de UMAS**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "pachuca" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_pachuca',
 'df_financiamientos_2020_pachuca',
 'df_financiamientos_2021_pachuca',
 'df_financiamientos_2022_pachuca',
 'df_financiamientos_2023_pachuca',
 'df_financiamientos_2024_pachuca']

## 2022

In [2]:
finance = df_financiamientos_2022_pachuca[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

41
(1388, 2)


In [3]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\2444930754.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2' '5' '6' ... '0' '0' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)


,id,ingresos_rango
0,1,2.61-4.00
1,2,9.01-12.00
2,3,> 12.00
3,4,> 12.00
4,5,4.01-6.00
...,...,...
1383,1384,0
1384,1385,0
1385,1386,0
1386,1387,0


In [4]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']

# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']

# Calcular los porcentajes
total_registros = len(finance)
#conteo_ingresos['porcentajes'] = (conteo_ingresos['conteo'] / total_registros) * 100
conteo_ingresos['porcentajes'] = round((conteo_ingresos['conteo'] / total_registros) * 100, 2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,379,28.14
1,6.01-9.00,210,15.59
2,4.01-6.00,199,14.77
3,≤ 2.60,199,14.77
4,2.61-4.00,190,14.11
5,9.01-12.00,170,12.62


In [5]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
3,≤ 2.60,199,14.77
4,2.61-4.00,190,14.11
2,4.01-6.00,199,14.77
1,6.01-9.00,210,15.59
5,9.01-12.00,170,12.62
0,> 12.00,379,28.14


In [6]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    xaxis=dict(title='UMAS',),
    yaxis=dict(
        title='% Créditos', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2022', carpeta='assets\graficas')
fig.show()

<>:32: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
<>:32: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\9989650.py:32: SyntaxWarning: invalid escape sequence '\g'
  def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\9989650.py:47: SyntaxWarning: invalid escape sequence '\g'
  guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2022', carpeta='assets\graficas')


In [7]:
14.77+14.11+14.77+15.59+12.62+28.14

100.0

## 2023

In [8]:
finance = df_financiamientos_2023_pachuca[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

56
(1399, 2)


In [9]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\2444930754.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '6' ... '4' '0' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



,id,ingresos_rango
0,1,≤ 2.60
1,2,2.61-4.00
2,3,> 12.00
3,4,> 12.00
4,5,9.01-12.00
...,...,...
1394,1395,9.01-12.00
1395,1396,4.01-6.00
1396,1397,6.01-9.00
1397,1398,0


In [10]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']
# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
conteo_ingresos['porcentajes'] = round((conteo_ingresos['conteo'] / total_registros) * 100, 2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,317,23.60
1,6.01-9.00,241,17.94
2,4.01-6.00,206,15.34
3,2.61-4.00,196,14.59
4,9.01-12.00,196,14.59
5,≤ 2.60,187,13.92


In [11]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
5,≤ 2.60,187,13.92
3,2.61-4.00,196,14.59
2,4.01-6.00,206,15.34
1,6.01-9.00,241,17.94
4,9.01-12.00,196,14.59
0,> 12.00,317,23.60


In [12]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    xaxis=dict(title='UMAS',),
    yaxis=dict(
        title='% Créditos', # Título del eje y        
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2023', carpeta='assets\graficas')
fig.show()

<>:32: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:32: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\3906976152.py:32: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\3906976152.py:47: SyntaxWarning:

invalid escape sequence '\g'



## 2024

In [13]:
finance = df_financiamientos_2024_pachuca[['id','ingresos_rango']]
registros_0 = finance[finance['ingresos_rango'] == 0].shape[0]
print(registros_0)
df_2022 = finance.shape
print(df_2022)
#finance_2022_pachuca = finance_2022_pachuca[finance_2022_pachuca['ingresos_rango'] != 0]

34
(792, 2)


In [14]:
# Convertir la columna 'ingresos_rango' a cadena de texto (str) 
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].astype(str)
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].astype(str)
# Reemplazar los valores de rango con el texto correspondiente utilizando replace
#finance_2022['ingresos_rango'] = finance_2022['ingresos_rango'].replace({
finance.loc[:, 'ingresos_rango'] = finance['ingresos_rango'].replace({
    
    '0': '0',
    '1': '≤ 2.60',
    '2': '2.61-4.00',
    '3': '4.01-6.00',
    '4': '6.01-9.00',
    '5': '9.01-12.00',
    '6': '> 12.00',
})
finance

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\2444930754.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['6' '4' '5' '5' '6' '1' '6' '4' '4' '6' '6' '3' '3' '6' '1' '3' '6' '2'
 '3' '4' '0' '1' '2' '3' '4' '5' '6' '0' '1' '2' '3' '4' '5' '6' '4' '1'
 '4' '1' '2' '6' '6' '1' '3' '4' '5' '6' '2' '6' '6' '4' '5' '6' '6' '4'
 '5' '2' '4' '5' '6' '6' '4' '5' '5' '3' '4' '6' '4' '6' '5' '2' '2' '6'
 '3' '4' '5' '2' '5' '5' '5' '1' '1' '4' '6' '1' '2' '2' '3' '4' '0' '0'
 '0' '1' '1' '4' '2' '6' '6' '1' '6' '6' '4' '6' '1' '2' '3' '4' '0' '1'
 '2' '3' '4' '5' '6' '0' '0' '1' '4' '0' '1' '2' '3' '4' '5' '0' '1' '4'
 '5' '2' '2' '2' '4' '4' '5' '1' '2' '4' '5' '5' '6' '6' '2' '3' '1' '3'
 '5' '6' '2' '6' '1' '3' '3' '4' '5' '6' '6' '6' '1' '3' '3' '3' '2' '3'
 '4' '5' '5' '6' '6' '5' '6' '4' '4' '3' '5' '6' '5' '2' '3' '4' '5' '6'
 '5' '3' '3' '2' '4' '3' '5' '2' '1' '1' '4' '4' '5' '3' '4' '0' '1' '1'
 '6' '4' '5'

,id,ingresos_rango
0,1,> 12.00
1,2,6.01-9.00
2,3,9.01-12.00
3,4,9.01-12.00
4,5,> 12.00
...,...,...
787,788,9.01-12.00
788,789,9.01-12.00
789,790,0
790,791,0


In [15]:
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['ingresos_rango'] != '0']

# Calcular el conteo de cada rango de ingresos
conteo_ingresos = finance['ingresos_rango'].value_counts().reset_index()
conteo_ingresos.columns = ['rango_ingresos', 'conteo']
# Calcular los porcentajes
total_registros = len(finance)
conteo_ingresos['porcentajes'] = round((conteo_ingresos['conteo'] / total_registros) * 100,2)
conteo_ingresos

,rango_ingresos,conteo,porcentajes
0,> 12.00,159,20.98
1,4.01-6.00,144,19.00
2,6.01-9.00,133,17.55
3,2.61-4.00,116,15.30
4,9.01-12.00,111,14.64
5,≤ 2.60,95,12.53


In [16]:
# Definir el diccionario de orden de rangos de ingresos
orden_rangos = {
    '≤ 2.60': 1,
    '2.61-4.00': 2,
    '4.01-6.00': 3,
    '6.01-9.00': 4,
    '9.01-12.00': 5,
    '> 12.00': 6
}

# Añadir una columna para el orden de los rangos
conteo_ingresos['orden'] = conteo_ingresos['rango_ingresos'].map(orden_rangos)
# Ordenar el DataFrame por el orden de los rangos
conteo_ingresos = conteo_ingresos.sort_values(by='orden')
# Eliminar la columna de orden
conteo_ingresos = conteo_ingresos.drop(columns=['orden'])
conteo_ingresos

,rango_ingresos,conteo,porcentajes
5,≤ 2.60,95,12.53
3,2.61-4.00,116,15.30
1,4.01-6.00,144,19.00
2,6.01-9.00,133,17.55
4,9.01-12.00,111,14.64
0,> 12.00,159,20.98


In [19]:
labels = conteo_ingresos['rango_ingresos']
values = conteo_ingresos['porcentajes']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos en UMAS (2022)',
    #title_font_size = 22,
    xaxis=dict(title='UMAS',),
    yaxis=dict(
        title='% Créditos', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_creditos_umas_2024', carpeta='assets\graficas')
fig.show()

<>:32: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:32: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\3917510289.py:32: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5244\3917510289.py:47: SyntaxWarning:

invalid escape sequence '\g'

